<a href="https://colab.research.google.com/github/samjurassic/datascience-demo/blob/main/Kaggle_House_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
# Load libraries
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import pydotplus

from IPython.display import Image
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
from sklearn.tree import export_graphviz
from io import StringIO
from typing import List

In [34]:
train = pd.read_csv("train.csv")

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train[train.describe().columns].corr().sort_values("SalePrice").index

In [ ]:
train[["LotArea", "1stFlrSF", "SalePrice"]].corr()

In [55]:
X = train[['GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual']]
y = train["SalePrice"]

clf = DecisionTreeRegressor(max_depth=5, criterion="squared_error")

In [ ]:
clf.fit(X, y)
y_pred = clf.predict(X)

print(f"RMSE LOG: {metrics.mean_squared_log_error(y_true=y, y_pred=y_pred)}")

In [ ]:
# graph decision tree
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,
                filled=True, rounded=True,
                special_characters=True,feature_names = X.columns)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png('tree.png')
Image(graph.create_png())


In [124]:
model_cols = ['KitchenAbvGr', 'EnclosedPorch', 'MSSubClass', 'OverallCond', 'YrSold',
       'LowQualFinSF', 'Id', 'MiscVal', 'BsmtHalfBath', 'BsmtFinSF2',
       '3SsnPorch', 'MoSold', 'PoolArea', 'ScreenPorch', 'BedroomAbvGr',
       'BsmtUnfSF', 'BsmtFullBath', 'LotArea', 'HalfBath', 'OpenPorchSF',
       '2ndFlrSF', 'WoodDeckSF', 'LotFrontage', 'BsmtFinSF1', 'Fireplaces',
       'MasVnrArea', 'GarageYrBlt', 'YearRemodAdd', 'YearBuilt',
       'TotRmsAbvGrd', 'FullBath', '1stFlrSF', 'TotalBsmtSF', 'GarageArea',
       'GarageCars', 'GrLivArea', 'OverallQual']

# Use train_test_split to split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    train[model_cols],
    train["SalePrice"], test_size=0.9, random_state=15)

In [ ]:
gb = HistGradientBoostingRegressor(loss="squared_error", learning_rate=0.01, max_depth=2, max_iter=200)
gb.fit(X_train, y_train)

In [ ]:
gb_pred = gb.predict(X_test)
print(f"RMSE LOG: {metrics.mean_squared_log_error(y_true=y_test, y_pred=gb_pred)}")

### Make test set predictions and submission file

In [ ]:
test = pd.read_csv("test.csv")
test.head()

In [ ]:
test.info()

In [ ]:
test_x = test[model_cols]

test_x[test_x.isna().any(axis=1)]

In [116]:
# test_x.describe().T["count"]

In [130]:
y_pred = gb.predict(test_x)

In [131]:
output_df = pd.concat([test["Id"], pd.Series(y_pred).rename("SalePrice")], axis=1)
output_df.to_csv("sam_predictions_gb.csv", index=False)

In [ ]:
output_df.head()